In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

研究目的：

1 了解客户流失的大体状况   
2 探究是什么原因造成了用户流失。如：流失的客户有哪些共同点；使用哪些服务的用户流失较为严重

**背景说明：**

客户流失是电信行业最大的问题之一。客户流失率通常作为公司的一个关键业务指标，因为留住一个现有客户的成本远远低于获得一个新客户的成本。所以对于公司来说，恢复的长期客户比新招募的客户更有价值。电信客户流失预测有助于公司发现潜在的流失客户，并及时采取相应措施，对其进行赢回。此处的‘流失’主要针对自愿流失客户，即自主停止服务或切换到其他公司。

数据说明：

这个数据集包含电信客户的多种信息，以及他们是否在上个月流失了。数据中，每一行代表一个客户，每一列为该客户的不同属性。这些属性主要包括客户订购的服务、帐户信息和个人信息。我们将使用Python和Seaborn库结合特征工程、逻辑回归、随机森林来进行数据分析。

字段说明： 

* customerID：      用户ID
* gender：     用户性别 (Female, Male)
* SeniorCitizen：      是否为老年用户 (1: 是，0: 不是)
* Partner：      是否有配偶 (Yes, No)
* Dependents ：      是否有被扶养人 (Yes, No)
* tenure：      在网时长 (0-72个月)
* PhoneService：      是否订购电话服务 (Yes, No)
* MultipleLines ：     是否有多重线路(Yes, No, No phoneservice)
* InternetService ：     客户的互联网服务提供商 (No, DSL，fiber optic)
* OnlineSecurity  ：    是否订购网络安全服务 (Yes，No，No internetserive)
* OnlineBackup  ：    是否订购在线备份服务 (Yes，No，No internetserive)
* DeviceProtection  ：    是否有设备保护功能 (Yes，No，No internetserive)
* TechSupport ：     是否订购技术支持 (Yes，No，No internetserive)
* StreamingTV  ：    是否使用在线视频 (Yes，No，No internetserive)
* StreamingMovies  ：    是否使用在线电影 (Yes，No，No internetserive)
* Contract    ：  缴费周期 (月, 年,两年)
* PaperlessBilling ：     是否开通电子账单 (Yes, No)
* PaymentMethod   ：   付款方式 (bank transfer，credit card，electronic check，mailed check)
* MonthlyCharges  ：    每月消费金额
* TotalCharges ：     消费总金额
* Churn  ：    [标签]是否为流失客户 (Yes, No)

# 数据总览
拿到数据进行分析观察，以对数据进行大致了解。下面开始数据整体探索。

In [ ]:
"""导入基本库"""
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
"""导入数据"""
churn_raw = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
"""查看数据属性"""
churn_raw.info()  # TotalCharges 含有缺失值

In [ ]:
churn_raw.shape

结论:  
1 数据集有 7043 个样本,21 个特征,没有缺失值.  
2 TotalCharges 为 object 类型的数据(本应为 float64),说明该特征的数据中含有异常.  
3 标签为 Churn  

In [ ]:
""" 统计每个特征有多少个类别 """
for i,name in enumerate(churn_raw.columns):
    
    name_sum = churn_raw[name].value_counts().shape[0] 
    print("{}、{}      特征类别个数是：{}".format(i + 1, name, name_sum))

In [ ]:
""" 数据统计 """
churn_raw.describe()

**结论**  
    1 customerID 为 7043 个,表示没有重复样本.  
    2 TotalCharges 为 object 类型数据,此处 进行数据统计时并未纳入.  
    3 客户主要群体为 非老年用户  
    4 tenure 和 MonthlyCharges 分布较为均匀,未发现异常值  

# 数据预处理
总体按照连续、离散特征顺序来进行特征探索

## 无关特征去除 异常数据处理

In [ ]:
"""删除对分析无关的 用户ID"""
churn_raw = churn_raw.drop('customerID', axis=1)

In [ ]:
"""查找 TotalCharges 中的异常数据:发现异常值为空格,缺失样本占比为0.16%, 删除含有空值的样本"""
churn_raw['TotalCharges'] = churn_raw['TotalCharges'].replace(' ', np.nan).astype('float64')
churn_raw = churn_raw.dropna(how='any', axis=0)

In [ ]:
"""再次进行数据统计"""
churn_raw.describe()

**结论**  
1 此时数据含有 7032 个样本, 20 个特征  
2 TotalCharges 分布略有不均, 后续分析分布情况

## 分类特征转化为数值分索引
便于后续 逻辑回归建模等操作,使用数值索引表示分类特征

In [ ]:
replace_cols = ['gender', 'Partner', 'Dependents', 'OnlineSecurity', 'PhoneService', 'MultipleLines', 'OnlineBackup', 'DeviceProtection',
                'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling','Churn']
for i in replace_cols:
    churn_raw[i] = churn_raw[i].replace({'No internet service': 0})
    churn_raw[i] = churn_raw[i].replace({'No phone service': 0})

    churn_raw[i] = churn_raw[i].replace({'No': 0})
    churn_raw[i] = churn_raw[i].replace({'Yes': 1})

churn_raw['gender'] = churn_raw['gender'].replace({'Female': 1, 'Male': 0})

In [ ]:
# 将要进行get_dummies的数据保存一份
churn_mulcats = churn_raw[['InternetService','Contract','PaymentMethod']]
# 对多种分类情况的分类特征进行 one-hot
churn = pd.get_dummies(churn_raw)

## 连续特征离散化处理
将 tenure 特征做离散化处理. 便于观察不同在网年限的客户的流失情况

In [ ]:
"""根据 tenure 的分布情况, 以12个月为一段,将客户分箱分桶"""

bins = [0, 12, 24, 36, 48, 60, 100]  
labels = ['one_year', 'two_years','three_years','four_years','five_years','over_5_years']

# 按照给定区间 使用 pd.cut 将数据进行离散化
churn['tenure_label'] = pd.cut(churn['tenure'], bins=bins, labels=labels)

In [ ]:
"""再次确认没有缺失值"""
churn.isna().any().sum()
churn.shape

**此时**  
7032 个样本, 28 个特征,没有缺失值,除了 tenure_label 均为数值型数据

# 客户流失率初步统计分析

## 查看总流失率

In [ ]:
churn.Churn.value_counts()

In [ ]:
churn_rate = churn.Churn.value_counts(1)
labels = ['Churn: No','Churn: Yes']
colors = ['c', 'r']
plt.pie(churn_rate, labels=labels, colors=colors, autopct='%3.1f%%')

**总流失率**  
流失客户为 1869 人, 占总数的 26.6%, 超过1/4 

## 各变量和客户流失的相关性

### 所有特征的相关性总览

In [ ]:
corrmat=churn.corr()
f,ax=plt.subplots(figsize=(20,20))

# cmap='RdBu' 设置颜色
# 设置center数据时，如果有数据溢出，则手动设置的vmax、vmin会自动改变 vmax=1
sns.heatmap(corrmat,cmap='RdBu',square=True)

# 设置坐标轴标签字体大小
ax.tick_params(labelsize=18)
plt.title('Important Variables correlation map',fontsize=18)

如图所示  
正相关关系  
* TotalCharges/ Contract_Two year 均与 tenure
* MonthlyCharges 与 消费产品指标 大多呈正相关  

负相关关系
* Churn 与 tenure
* Contract_Month-to-month 与 tenure
* InternetService_No 与大部分特征成负相关,尤其是 MonthlyCharges

**结论**  
* tenure 可能是一个关键指示指标: 在网时间越长,客户消费相应增加,越倾向与签订两年期合同.越不易流失
* 按月缴费用户普遍服务期较短,可能新用户居多,也更易流失

### 流失标签与各个特征的相关性

In [ ]:
# 计算每一个特征和目标（流失）的相关系数，排序画图，试图从特征相关性这个维度找到重要特征
churn_corr = churn.corr()[['Churn']].sort_values(by='Churn').drop(['Churn']).reset_index()
churn_corr= churn_corr.rename(columns={'index':'col_labels','Churn':'corr_values'})

fig,ax=plt.subplots(figsize=(12,12))
ind = np.arange(churn_corr.shape[0])
rects=ax.barh(ind,churn_corr.corr_values.values,color='c', height=0.4)

ax.set_yticks(ind)  # y 轴刻度
ax.set_yticklabels(churn_corr.col_labels.values,rotation='horizontal',fontsize=12)  # y 轴的标签
ax.set_xlabel('Correlation coefficient',fontsize=12)
ax.set_title('Correlation coefficient of the variables',fontsize=12)
plt.grid(linestyle='--', alpha=0.5)

**与流失成正相关的前五个的特征**
* Contract_Month-to-month
* InternetService_Fiber optic
* PaymentMethod_Electronic check
* MonthlyCharges【月消费越高, 容易流失?】
* PaperlessBilling  

**与流失成负相关的前五个的特征**
* tenure
* Contract_Two year
* InternetService_No
* TotalCharges
* Contract_One year

**结论**
* 正负相关的特征并不局限于订购的服务、帐户信息和个人信息中的某类,而是交叉出现,所以后续将对用户的这三大类特征分别进行研究

In [ ]:
# 订购的服务
order_info = ['PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
              'TechSupport', 'StreamingTV', 'StreamingMovies', 'InternetService_DSL',
              'InternetService_Fiber optic', 'InternetService_No']

# 帐户信息
account_info = ['tenure', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Contract_Month-to-month',
                'Contract_One year', 'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
                'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check',
                'PaymentMethod_Mailed check']

# 个人信息
per_info = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']

print('order_info 字段个数:', len(order_info))
print('account_info 字段个数:', len(account_info))
print('per_info 字段个数:', len(per_info))

## 用户服务订购情况与流失与否的相关性研究

In [ ]:
for i, name in enumerate(order_info):
    sns.countplot(data=churn, y=name, hue='Churn')
    plt.tick_params(labelsize=14)
    plt.xlabel('count', size=14)
    plt.ylabel(name, size=14)  # 感觉这方法好笨,sns应该能一并设置吧,怎样自定义图例显示内容呢???

    length = churn.groupby(name)['Churn'].value_counts().shape[0]
    
    for p in range(length):
#         churn_num = 
        rate = churn.groupby(name)['Churn'].value_counts(1)[p]
        print(rate)

#         plt.text(p,100, str(rate),fontsize=12)  # 碍于技术上的难题,还没标到图上...
    plt.show()

**结论**
* 订购 MultipleLines, StreamingTV, StreamingMovies, 以及使用光纤(InternetService_Fiber optic)的用户**流失率比平均流失水平高**,需要检查是不是光纤速度较慢,在线电视电影是否卡顿或资源不足导致客户流失.对于 StreamingTV, StreamingMovies 这些娱乐项目,用户经常是哪个运营商有想看的内容就在使用哪个运营商,所以提高用户体验的方式除了流畅度,还要加大购买资源的力度
* 订购 OnlineSecurity, TechSupport, 使用 InternetService_DSL 及不使用互联网的用户流失率 明显**低于平均流失水平**, 推进OnlineSecurity, TechSupport在客户中的普及率(也不能一味推进把,是不是有这类需求的客户是哪一类人? 努力招募这种类型的人),继续完善 DSL 网络服务,多向客户推荐此网络
* 订购 OnlineBackup, DeviceProtection 的用户流失率相对平均流失水平略低. 可以推测 使用 OnlineSecurity,TechSupport,OnlineBackup 这些客户本来就是较为稳定的客户,除非与其他电电信公司服务差距比较大,否则从自身经验来看,这些服务内容,并不是经常在不同的运营商之间迁移.所以要观察同行的动态,避免出现差距.

## 帐户信息与流失与否的相关性
连续型变量和分类型变量分别处理

In [ ]:
"""连续型变量"""
account_con = ['tenure','MonthlyCharges','TotalCharges']

"""分类型变量"""
account_cat = ['PaperlessBilling','Contract_Month-to-month','Contract_One year','Contract_Two year',
               'PaymentMethod_Bank transfer (automatic)','PaymentMethod_Credit card (automatic)',
               'PaymentMethod_Electronic check','PaymentMethod_Mailed check']

### 连续型变量的处理

In [ ]:
"""核密度估计"""
# 在没有已测得的样本分布的情况下，估计在取值点处的概率密度函数

def kdeplot(feature):
    plt.figure(figsize=(9, 4))
    plt.title("KDE for {}".format(feature))
    ax0 = sns.kdeplot(churn[churn['Churn'] == 0][feature].dropna(), color= 'c', label= 'Churn: No')
    ax1 = sns.kdeplot(churn[churn['Churn'] == 1][feature].dropna(), color= 'r', label= 'Churn: Yes')

for i in account_con:
    kdeplot(i)

* 新入网的用户 流失率最高,流失比例几乎是未流失的两倍,但随着在网时长的增加,未流失用户趋于平稳,流失比率明显降低.
* 流失客户在月资费处于80-100区间内比例最高,未流失客户主要处于低月资费与高月资费,中间资费人数比较少,要留意低月资费区间的客户是否为"僵尸"客户,活跃度较低.
* 总消费越高的区间,未流失客户占比较高.而总消费较低的区间,流失客户占比相对较高.
* 综上: 要笼络好在网时间长,累计消费额较高的用户, 并使用发放问卷等形式了解月消费额较高的用户的不同需求, 推出相应的活动.争取降低 高月资费用户的流失率.

In [ ]:
g = sns.PairGrid(churn, y_vars=account_con, x_vars=account_con, height=3, hue="Churn", aspect=1.1)
ax = g.map(plt.scatter, alpha=0.6, s = 10)

* 随在网时长的增加, 用户消费金额总计升高, 但月资费较高群体主要集中在新入网用户. (为什么新入网的人会选择高资费? 这群人是什么人?)
* 因为新入网不久,所以月资费较高的人在消费金额总计较低的水平分布较高,但消费金额总计较高的用户中,也分布有较多的月资费较高的用户. 月资费居中的用户, 相比其他用户,在高消费总金额中密度较低

### 分类型变量的处理

In [ ]:
for i, name in enumerate(account_cat):
    sns.countplot(data=churn, y=name, hue='Churn')
    plt.tick_params(labelsize=14)
    plt.xlabel('count', size=14)
    plt.ylabel(name, size=14)  # 感觉这方法好笨,sns应该能一并设置吧,怎样自定义图例显示内容呢???

    length = churn.groupby(name)['Churn'].value_counts().shape[0]
    
    for p in range(length):
#         churn_num = 
        rate = churn.groupby(name)['Churn'].value_counts(1)[p]
        print(rate)

#         plt.text(p,100, str(rate),fontsize=12)  # 碍于技术上的难题,还没标到图上
    plt.show()

* 每个特征都有较为明显的区分度,其中客户流失较为严重群体为开通电子账单(PaperlessBilling),按月缴费(Contract_Month-to-month),使用电子支票(PaymentMethod_Electronic check)的用户
* 流失率较低的群体,主要为 按年缴费或两年期缴费, 使用银行卡转账及信用卡或邮寄支票的客户
* 公司可以通过促使用户按年或两年缴费来提高用户粘性(不同的支付方式是不是标志着不同的人群?年轻人使用银行卡?老人用电子支付等?瞎猜)

## 用户个人状况与流失的相关性及客户价值探索
这里直接说相关性是不是不太严谨??

In [ ]:
per_info = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
for i, name in enumerate(per_info):
    sns.countplot(data=churn, y=name, hue='Churn')
    plt.tick_params(labelsize=14)
    plt.xlabel('count', size=14)
    plt.ylabel(name, size=14)  # 感觉这方法好笨,sns应该能一并设置吧,怎样自定义图例显示内容呢???

    length = churn.groupby(name)['Churn'].value_counts().shape[0]
    
    for p in range(length):
#         churn_num = 
        rate = churn.groupby(name)['Churn'].value_counts(1)[p]
        print(rate)

#         plt.text(p,100, str(rate),fontsize=12)  # 碍于技术上的难题,还没标到图上
    plt.show()

* 流失客户在性别上没有体现出差异性
* 老年人流失率极高
* 没有配偶的客户流失率较高(年轻人?)
* 没有子女的客户流失率较高(应该还是年轻人吧)

In [ ]:
"""订购不同服务的不同年龄段的用户的流失情况"""

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

services = ['MultipleLines','PhoneService','OnlineSecurity','OnlineBackup',
                   'DeviceProtection','TechSupport','StreamingTV','StreamingMovies']

# services = ['MultipleLines','OnlineSecurity','OnlineBackup',
#                    'DeviceProtection','TechSupport','StreamingTV','StreamingMovies']

senior_stats = list()
senior_churn_stats = list()
junior_stats = list()
junior_churn_stats = list()

for name in services:
    
    # 老年人的流失情况
    senior = (churn.groupby([name])['SeniorCitizen'].value_counts()[3])/churn.shape[0]
    senior_churn = (churn.groupby([name,'SeniorCitizen'])['Churn'].value_counts()[7])/churn.shape[0]
    
    # 非老年人的流失情况
    junior = (churn.groupby([name])['SeniorCitizen'].value_counts()[2])/churn.shape[0]
    junior_churn = (churn.groupby([name,'SeniorCitizen'])['Churn'].value_counts()[5])/churn.shape[0]

    senior_stats.append(senior)    
    senior_churn_stats.append(senior_churn)
    junior_stats.append(junior)    
    junior_churn_stats.append(junior_churn)
    
labels = np.array(services)

# 绘制雷达网格
angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False)  # 创建等差数列
angles = np.concatenate((angles, [angles[0]]))

senior_stats = np.concatenate((senior_stats, [senior_stats[0]]))
senior_churn_stats = np.concatenate((senior_churn_stats, [senior_churn_stats[0]]))

junior_stats = np.concatenate((junior_stats, [junior_stats[0]]))
junior_churn_stats = np.concatenate((junior_churn_stats, [junior_churn_stats[0]]))


fig = plt.figure(figsize=(6,6),dpi=100)
ax = fig.add_subplot(111, polar=True)
ax.plot(angles, senior_stats, '-', linewidth=1,color='blue', alpha=0.01)
ax.fill(angles, senior_stats, color='blue',alpha=0.5)  # alpha 透明度

ax.plot(angles, senior_churn_stats, '-', linewidth=1,color='gold', alpha=0.01)
ax.fill(angles, senior_churn_stats, color='gold',alpha=0.5)

ax.plot(angles, junior_stats, '-', linewidth=1,color='green')

ax.plot(angles, junior_churn_stats, '-', linewidth=1,color='red')

# 设置中文字体
# font = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf", size=14)
ax.set_thetagrids(angles * 180/np.pi, labels)  # , FontProperties=font
plt.show()

**结论**
* 开通各项服务的顾客,基本只占使用电话服务的一半 
* 非老年人在用户中占主体地位,各项服务流失人数基本与老年用户总数持平.
* [感觉这图大部分空间都浪费了]

In [ ]:
"""不同支付方式 不同年龄段的用户的消费情况"""

sns.catplot(x="SeniorCitizen", y="MonthlyCharges", hue="Churn",
            kind="violin", split=True, data=churn)

sns.catplot(x="SeniorCitizen", y="TotalCharges", hue="Churn",
            kind="violin", split=True, data=churn)

**结论**
* 月资费角度: 非老年人在低月资费未流失客户占比较高,而高月资费处,流失客户相对较高; 老年人在高月资费处分布较高,流失与未流失比率相当
* 总消费金额角度: 老年人在总消费金额较高的区间,忠诚客户占比比较高
* 综上:即使老年客户在客户总数中占比不不高,但消费能力较强,要积极挽回

In [ ]:
"""有无配偶及子女的客户的消费能情况"""


sns.boxplot(x="Partner", y="MonthlyCharges",
            hue="Churn", data=churn)  # palette=["m", "g"],  色调
# sns.despine(offset=10, trim=True)  # 边框设置
plt.show()

sns.boxplot(x="Partner", y="TotalCharges",
            hue="Churn", data=churn)

In [ ]:
sns.boxplot(x="Partner", y="tenure",
            hue="Churn", data=churn)

**结论**
* 月资费角度: 有配偶的客户在高月资费区间人数略多,但同时流失顾客也主要分布在高月资费区间
* 总消费额角度: 客户中有配偶的人数较多. 没有配偶的的客户在总消费金额较低的区间比较集中有配偶的顾客总消费较低的人更易流失.
* 在网时长: 无论有无配偶,低在网时长的客户都更易流失. 没有配偶的人,无论是流失客户还是忠诚客户,在网时长都相对较低, 可能是较为年轻的人刚入网的原因? 这也导致了没有配偶的人总消费金额较低. 但这部分人成长空间较大,适宜长期发展. 结合实际,刚刚入网的人可能稳定性较低,可以提供新人续费优惠等活动,挽留住次新客户; 值得注意的是有配偶客户流失人数几乎和忠诚客户占比相当,应引起注意,查找原因. 

In [ ]:
customer_cat = ['gender','SeniorCitizen','Partner','Dependents', 'tenure_label']
churn_all = pd.concat([churn_mulcats, churn],axis=1)
for i in customer_cat:
    sns.countplot(data=churn_all, x="Contract", hue=i)
    plt.show()

**结论**
* 缴费周期在不同性别间没有分化
* 老年人更偏向于短周期的缴费方式
* 有伴侣的顾客偏向长周期的缴费方式,稳定性更高
* 没有子女的顾客更偏向于月缴费.
* 在网时间越长,客户越偏向两年缴费的方式
* 综上: 有配偶和子女的顾客在缴费方式上偏向长周期,更为稳定,老年人的选择短周期容易理解; 年轻人可能更加活跃在,偏向月缴费,而这部分人也存在走向稳定家庭生活的趋势,长远打算: 抓住这部分人,就更可能拥有一批稳定客户,虽然这部分人现阶段消费能力可能不高.

# 逻辑回归分析

In [ ]:
"""去除含有缺失值的样本"""
churn = churn.dropna(how='any', axis=0)
"""查看样本是否均衡"""
churn.groupby(['Churn']).count()  # 相差两倍，不需要做样本均衡处理

In [ ]:
"""选择特征值和目标值"""
x = churn.drop(['tenure_label','Churn'], axis=1)
y = churn.Churn

* 除了入网时长 tenure 数值相对较分散,其他特征均为 0,1 取值,为了评估 tenure 对模型的影响,分别尝试原数据集直接建模、数据标准化后建模、数据归一化后建模 三种方式

## 原数据集直接建模

In [ ]:
# 划分训练集 测试集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=11)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
result = lr_model.fit(x_train, y_train)

In [ ]:
# 回归系数
print(lr_model.coef_)
# 截距
print(lr_model.intercept_)

### 模型评估

In [ ]:
# 使用模型进行预测
y_pred_train = lr_model.predict(x_train)
y_pred_test = lr_model.predict(x_test)

# # 查看得分
# from sklearn.metrics import accuracy_score
# train_score = accuracy_score(y_train, y_pred_train)
# test_score = accuracy_score(y_test, y_pred_test)
# # 得分就是准确率
# print('训练集预测得分：{}'.format(train_score))
# print('测试集预测得分：{}'.format(test_score))

# # 查看召回率
# from sklearn.metrics import auc, roc_curve
# fpr, tpr, _ = roc_curve(y_train, y_pred_train)
# roc_auc = auc(fpr,tpr)
# print('召回率为：{}'.format(roc_auc))

from sklearn.metrics import classification_report
ret = classification_report(y_test, y_pred_test, labels=(0,1), target_names=('忠诚', '流失'))
print(ret)

## 标准化后进行逻辑回归

In [ ]:
from sklearn.preprocessing import StandardScaler
z_scaler = StandardScaler()
z_x = pd.DataFrame(z_scaler.fit_transform(x),columns=x.columns)

zx_train, zx_test, zy_train, zy_test = train_test_split(z_x, y, test_size=0.3, random_state=11)

zlr_model = LogisticRegression()
z_result = zlr_model.fit(zx_train, zy_train)

In [ ]:
# 回归系数
print(zlr_model.coef_)
# 截距
print(zlr_model.intercept_)

### 模型评估

In [ ]:
# 使用模型进行预测
zy_pred_train = zlr_model.predict(zx_train)
zy_pred_test = zlr_model.predict(zx_test)

from sklearn.metrics import classification_report
ret = classification_report(zy_test, zy_pred_test, labels=(0,1), target_names=('忠诚', '流失'))
print(ret)

## 归一化后进行逻辑回归

In [ ]:
from sklearn.preprocessing import MinMaxScaler
m_scaler = MinMaxScaler()
m_x = pd.DataFrame(m_scaler.fit_transform(x), columns=x.columns)

In [ ]:
# 划分测试集、训练集
mx_train, mx_test, my_train, my_test = train_test_split(m_x, y, test_size=0.3, random_state=11)
# 构建逻辑回归模型
mlr_model = LogisticRegression()
m_result = mlr_model.fit(mx_train, my_train)
# 回归系数
print(mlr_model.coef_)
# 截距
print(mlr_model.intercept_)

### 模型评估

In [ ]:
# 使用模型进行预测
my_pred_train = mlr_model.predict(mx_train)
my_pred_test = mlr_model.predict(mx_test)

from sklearn.metrics import classification_report
ret = classification_report(my_test, my_pred_test, labels=(0,1), target_names=('忠诚', '流失'))
print(ret)

## 本章结论
- 经过标准化处理的数据,在召回率与 f1 得分 略有些优势,但模型总体的稳健性不高,在预测流失客户上有相当大的偏差
- 可以尝试不同比例划分训练集测试集/均衡处理等方法继续尝试 [我在这暂时不想尝试了....]
- 也可能是数据集样本量不足导致训练效果不佳
- [查下别人都是怎么调整的]

In [ ]:
""" 经逻辑回归得到的各变量值的相关系数(这怎么解释啊,就是得到了这么个公式啊...拿去用好了,反正也不准哈哈哈哈)"""
z_coef = pd.DataFrame(zlr_model.coef_,columns=z_x.columns).T.sort_values(by=0)
z_coef = z_coef.rename(columns={ 0: 'coef_values'})

In [ ]:
z_coef.plot(kind='bar')

# 随机森林

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rx_train, rx_test, ry_train, ry_test = train_test_split(z_x, y, test_size=0.3, random_state=11)
rf_model = RandomForestClassifier(n_estimators=1000 , oob_score = True,
                                  random_state =11, max_features = "auto",
                                  max_leaf_nodes = 30)
rf_model.fit(rx_train, ry_train)

## 模型评估

In [ ]:
# 使用模型进行预测
from sklearn.metrics import accuracy_score

ry_pred_train = rf_model.predict(rx_train)
ry_pred_test = rf_model.predict(rx_test)

# 查看得分
print('训练集预测得分：{}'.format(accuracy_score(ry_train, ry_pred_train)))
print('测试集预测得分：{}'.format(accuracy_score(ry_test, ry_pred_test)))

# 查看importances
importances = rf_model.feature_importances_
print('importances：{}'.format(importances))

In [ ]:
from sklearn.metrics import classification_report
# ret = classification_report(ry_test, ry_pred_test, labels=(0,1), target_names=('忠诚', '流失'))
ret = classification_report(ry_test, ry_pred_test)

print(ret)  # 这个召回率有点丑... 结论同 第四章

In [ ]:
"""模型得到的指标"""
z_importances = pd.DataFrame([importances],columns=z_x.columns).T.sort_values(by=0)
z_importances = z_importances.rename(columns={ 0: 'importances'})
z_importances

In [ ]:
z_importances.plot(kind='bar')

# 结论
* 共有客户7043人, 流失客户为 1869 人, 占总数的 26.6%, 超过1/4

**易流失人群**
* 用户维度: 老人, 单身, 无子女, 在网时长小于一年的用户容易流失.
* 产品维度: 不开通技术性增值服务; 开通多线服务/光纤网络/娱乐性服务的客户
* 消费维度: 月消费80-100元, 选择月缴费, 使用Electronic check支付[可以继续分析一下什么人在用这种卡]
    
**建议**
* 对年轻人: 积极推出活动,稳住新人和次新人, 诱导其选择长期合同
* 对有配偶和子女及老年客户急需保持, (老年人是不容易签长合同了)
* 调查光纤(InternetService_Fiber optic)用户流失率高的原因
* 在娱乐资源 (StreamingTV, StreamingMovies) 上加大投入和服务,推广技术性增值服务, 留住更多客户.
* 推广 InternetService_DSL 的使用

**预测模型构建**
* 用户流失预测模型还需要进一步完善...